<a href="https://colab.research.google.com/github/tsimut/Final-Project/blob/master/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np

In [0]:
review=[]
rating=[]
name=[]

In [0]:
page_count = 0
while page_count <10:
    
    url = "https://www.consumeraffairs.com/travel/uber.html?page=%s" %(page_count)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    reviews=soup.find(id="reviews-container")
    
    comments=soup.find_all("div", class_="rvw-bd ca-txt-bd-2")
    for ct in comments:
        review.append(ct.find("p").get_text())
    
    names=reviews.find_all("div",class_="rvw-aut__inf")
    for n in names:
      name.append(n.find("strong",class_="rvw-aut__inf-nm").get_text())

    try:
      star=reviews.find_all("div",class_="rvw__hdr-stat")
      for st in star:
        rating.append(st.find("img")["data-rating"])
    except:
      continue
    time.sleep(0.05)  
    page_count += 1
    

In [4]:
Reviews = pd.DataFrame({
"Name": name,
"Rating": rating,
"Review":review
})


Reviews.head(20)

,Name,Rating,Review
0,"Dustin of St Louis, MO",4.0,I have had a few great experiences and a coupl...
1,"John of Duncansville, PA",5.0,"I’ve never used Lyft before, and the only cab ..."
2,"Pedro of Boston, Massachusetts",5.0,I've never had a negative experience with Uber...
3,"Barbara of Chicago, IL",4.0,I have always been skeptical about rideshare s...
4,"Jennifer of Philadelphia , PA",5.0,I have used Uber several times and so far I am...
5,"Yuge of Fayetteville, AR",1.0,"First ** time I called you, my drivers says, ""..."
6,"Ash of Delhi, Other",1.0,Uber does only lip service in term of driver q...
7,"Preeti of Irving, TX",1.0,"I requested a ride from Fort Worth, South Hend..."
8,"Ridhi of Chennai, Other",1.0,"Earlier, Uber was great. They used to provide ..."
9,"Lev of Bayside, NY",1.0,I have been using Uber as a rider from 2016 wi...


In [0]:
Sentiments=[]
for i in Reviews["Rating"]:
  x=float(i)
  if x <3:
    Sentiments.append("N")

  elif 3 < x < 6:
      Sentiments.append("P")
  else:
    Sentiments.append("O")
      
    

In [0]:
Reviews['Sentiments'] = Sentiments

In [7]:
Reviews.head(20)

,Name,Rating,Review,Sentiments
0,"Dustin of St Louis, MO",4.0,I have had a few great experiences and a coupl...,P
1,"John of Duncansville, PA",5.0,"I’ve never used Lyft before, and the only cab ...",P
2,"Pedro of Boston, Massachusetts",5.0,I've never had a negative experience with Uber...,P
3,"Barbara of Chicago, IL",4.0,I have always been skeptical about rideshare s...,P
4,"Jennifer of Philadelphia , PA",5.0,I have used Uber several times and so far I am...,P
5,"Yuge of Fayetteville, AR",1.0,"First ** time I called you, my drivers says, ""...",N
6,"Ash of Delhi, Other",1.0,Uber does only lip service in term of driver q...,N
7,"Preeti of Irving, TX",1.0,"I requested a ride from Fort Worth, South Hend...",N
8,"Ridhi of Chennai, Other",1.0,"Earlier, Uber was great. They used to provide ...",N
9,"Lev of Bayside, NY",1.0,I have been using Uber as a rider from 2016 wi...,N


In [0]:
punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'
Reviews['Review'] = Reviews['Review'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

In [0]:
# convert text to lowercase
Reviews['Review'] = Reviews['Review'].str.lower()

In [0]:
# remove numbers
Reviews['Review'] = Reviews['Review'].str.replace("[0-9]", " ")

In [11]:
# remove whitespaces
Reviews['Review'] = Reviews['Review'].apply(lambda x:' '.join(x.split()))
Reviews['Review']

0      i have had a few great experiences and a coupl...
1      i’ve never used lyft before, and the only cab ...
2      i've never had a negative experience with uber...
3      i have always been skeptical about rideshare s...
4      i have used uber several times and so far i am...
5      first time i called you, my drivers says, i ca...
6      uber does only lip service in term of driver q...
7      i requested a ride from fort worth, south hend...
8      earlier, uber was great. they used to provide ...
9      i have been using uber as a rider from with a ...
10     uber eats cancel order without refund. ubereat...
11     i took uber from igt to sector noida and it sh...
12     so my husband and i thought to let everybody k...
13     i was in montreal last week as it was raining....
14     launched app, was quoted for the mile ride hom...
15     i placed an order on uber eats and the food ne...
16     customer service is absolutely terrible. i hav...
17     on thursday i lost my wa

In [0]:
import spacy
nlp = spacy.load('en', disable=['parser', 'ner'])

# function to lemmatize text
def lemmatization(texts):
    output = []
    for i in texts:
        s = [token.lemma_ for token in nlp(i)]
        output.append(' '.join(s))
    return output

In [0]:
Reviews['Review'] = lemmatization(Reviews['Review'])


In [14]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
Reviews['Review'] = Reviews['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
Reviews['Review']

0      great experience couple bad experience . bad e...
1      -PRON- never use lyft , cab company highly ine...
2      -PRON- never negative experience uber . -PRON-...
3      always skeptical rideshare service news report...
4      use uber several time far happy company easy -...
5      first time call -PRON- , -PRON- driver say , m...
6      uber lip service term driver quality india . l...
7      request ride fort worth , south henderson stre...
8      earlier , uber great . -PRON- use provide prom...
9      use uber rider good passenger rating . never i...
10     uber eat cancel order without refund . ubereat...
11     take uber igt sector noida -PRON- show rent ap...
12     -PRON- husband think let everybody know -PRON-...
13     montreal last week -PRON- rain . want quick ri...
14     launch app , quote mile ride home , expect , -...
15     place order uber eat food never show charge an...
16     customer service absolutely terrible . recentl...
17     thursday lose -PRON- wal

In [0]:
# remove short words (length < 3)
Reviews["Review"] = Reviews["Review"].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))


In [17]:
#IDENTIFY TOP 10 OCCURING WORDS
freq = pd.Series(' '.join(Reviews['Review']).split()).value_counts()[:10]
freq

-PRON-     2047
uber        540
driver      315
use         177
ride        176
get         169
time        160
charge      121
take        119
service     110
dtype: int64

In [20]:
common_word=("uber,driver,use,ride,get,take")
Reviews['Review'] = Reviews['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in common_word))
Reviews['Review'].head()

0    great experience couple bad experience bad exp...
1    -PRON- never lyft cab company highly ineffecti...
2    -PRON- never negative experience -PRON- one do...
3    always skeptical rideshare service news report...
4    several time far happy company easy -PRON- set...
Name: Review, dtype: object

0    great experience couple bad experience bad exp...
1    -PRON- never use lyft cab company highly ineff...
2    -PRON- never negative experience -PRON- one do...
3    always skeptical rideshare service news report...
4    use several time far happy company easy -PRON-...
Name: Review, dtype: object